In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/EmotionalDamage

/content/gdrive/MyDrive/EmotionalDamage


In [ ]:
import sys, os  
import pandas as pd  
import numpy as np  

from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D  
from keras.losses import categorical_crossentropy  
from keras.optimizers import Adam  
from keras.regularizers import l2  
from keras.utils import np_utils 


In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/EmotionalDamage/fer2013.csv')  
X_train,train_y,X_test,test_y=[],[],[],[]  
for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    try:  
        if 'Training' in row['Usage']:  
           X_train.append(np.array(val,'float32'))  
           train_y.append(row['emotion'])  
        elif 'PublicTest' in row['Usage']:  
           X_test.append(np.array(val,'float32'))  
           test_y.append(row['emotion'])  
    except:  
        print(f"error occured at index :{index} and row:{row}") 


In [ ]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 30  
width, height = 48, 48  

X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)   
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)  

X_train -= np.mean(X_train, axis=0) #normalize dữ liệu giữa 0 và 1 
X_train /= np.std(X_train, axis=0)  

X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)  



In [ ]:
model = Sequential()  

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(Conv2D(64, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(Conv2D(128, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  

model.add(Flatten())  

#fully connected neural networks  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  

model.add(Dense(num_labels, activation='softmax'))  

# model.summary()  

#Compliling the model  
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy']) 


In [ ]:
model.summary


<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f2431f23730>>

In [ ]:
#Training the model  
model.fit(X_train, train_y,  
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, test_y),  
          shuffle=True)  


Epoch 1/30
449/449 [==============================] - 664s 1s/step - loss: 1.7260 - accuracy: 0.2924 - val_loss: 1.5289 - val_accuracy: 0.3965
Epoch 2/30
449/449 [==============================] - 668s 1s/step - loss: 1.4927 - accuracy: 0.4139 - val_loss: 1.3856 - val_accuracy: 0.4642
Epoch 3/30
449/449 [==============================] - 644s 1s/step - loss: 1.3890 - accuracy: 0.4617 - val_loss: 1.3195 - val_accuracy: 0.4976
Epoch 4/30
449/449 [==============================] - 638s 1s/step - loss: 1.3201 - accuracy: 0.4937 - val_loss: 1.2487 - val_accuracy: 0.5269
Epoch 5/30
449/449 [==============================] - 634s 1s/step - loss: 1.2787 - accuracy: 0.5094 - val_loss: 1.2610 - val_accuracy: 0.5196
Epoch 6/30
449/449 [==============================] - 630s 1s/step - loss: 1.2462 - accuracy: 0.5243 - val_loss: 1.2210 - val_accuracy: 0.5313
Epoch 7/30
449/449 [==============================] - 629s 1s/step - loss: 1.2154 - accuracy: 0.5350 - val_loss: 1.1913 - val_accuracy: 0.5456

In [ ]:
#Saving the  model 
fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(path_to/fer_json)  
model.save_weights("/content/gdrive/MyDrive/EmotionalDamage/fer.h5")  
